## Exercice n°1 (1/2 h)




Dans le modèle ci-dessous, pouvez-vous préciser :
- la taille du champ réceptif associé à un pixel d'une carte de caractéristique en sortie de enc3, par un calcul théorique.
- passer ce modèle sur un champ ne contenant que des zéros sauf pour une composante. En déduire la taille du champ réceptif empirique en sortie du modèle.


In [ ]:
import torch
import torch.nn as nn

class MyNN(nn.Module):
    def __init__(self, in_ch=3, out_ch=1, base=16):
        super().__init__()

        # 2 Conv2D
        self.enc1 = nn.Sequential(
            nn.Conv2d(in_ch, base, kernel_size=3, stride=1, padding=1, bias=False),
            nn.ReLU(),
            nn.Conv2d(base, base, kernel_size=3, stride=1, padding=1, bias=False),
            nn.ReLU(),
        )

        # 2 Conv2D, first with stride=2
        self.enc2 = nn.Sequential(
            nn.Conv2d(base, base * 2, kernel_size=3, stride=2, padding=1, bias=False),
            nn.ReLU(),
            nn.Conv2d(base * 2, base * 2, kernel_size=3, stride=1, padding=1, bias=False),
            nn.ReLU(),
        )

        # again 2 Conv2D, first with stride=2
        self.enc3 = nn.Sequential(
            nn.Conv2d(base * 2, base * 4, kernel_size=3, stride=2, padding=1, bias=False),
            nn.ReLU(),
            nn.Conv2d(base * 4, base * 4, kernel_size=3, stride=1, padding=1, bias=False),
            nn.ReLU(),
        )

        # alternance Conv2D / ConvTranspose2D
        self.mid = nn.Conv2d(base * 4, base * 4, kernel_size=3, padding=1, bias=False)

        self.up1 = nn.ConvTranspose2d(base * 4, base * 2, kernel_size=2, stride=2, bias=False)
        self.dec1 = nn.Conv2d(base * 2, base * 2, kernel_size=3, padding=1, bias=False)

        self.up2 = nn.ConvTranspose2d(base * 2, base, kernel_size=2, stride=2, bias=False)
        self.dec2 = nn.Conv2d(base, base, kernel_size=3, padding=1, bias=False)

        self.head = nn.Conv2d(base, out_ch, kernel_size=1, bias=False)

    def forward(self, x):
        x = self.enc1(x)
        x = self.enc2(x)
        x = self.enc3(x)

        x = torch.relu(self.mid(x))

        x = self.up1(x)
        x = torch.relu(self.dec1(x))

        x = self.up2(x)
        x = torch.relu(self.dec2(x))

        return self.head(x)